In [1]:
from cloudinit.config.cc_disk_setup import device_type
from datasets import load_dataset
import torch
print(torch.__version__)
print(torch.cuda.is_available())
import torch.nn as nn
# from transformers.models.gpt2.modeling_gpt2 import GPT2Attention

2.5.1+cu124
True


In [3]:
dataset = load_dataset("wikitext", "wikitext-103-raw-v1")

In [2]:
from GPT2CustomAttention import CustomGPT2Attention, CustomGPT2FlashAttention
from data_loader import preprocess_text, split_in_sequence

In [3]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
device = "cuda"
model_id = "openai-community/gpt2"
model_vanilla = GPT2LMHeadModel.from_pretrained(model_id)
model_flash = GPT2LMHeadModel.from_pretrained(model_id)
tokenizer = GPT2TokenizerFast.from_pretrained(model_id)

config = model_flash.config
config

GPT2Config {
  "_attn_implementation_autoset": true,
  "_name_or_path": "openai-community/gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.46.2",
  "use_cache": true,
  "vocab_size": 50257
}

In [12]:
for block in model_vanilla.transformer.h:
    c_attn_weight = block.attn.c_attn.weight.clone()
    c_attn_bias = block.attn.c_attn.bias.clone()
    c_proj_weight = block.attn.c_proj.weight.clone()
    c_proj_bias = block.attn.c_proj.bias.clone()
    
    q_weight, k_weight, v_weight = torch.chunk(c_attn_weight, chunks=3, dim=1)
    q_bias, k_bias, v_bias = torch.chunk(c_attn_bias, chunks=3, dim=0)

    custom_attn = CustomGPT2Attention(nx=config.n_embd, n_ctx=config.n_ctx, config=config)
    custom_attn.q_attn.weight.data = q_weight.T.clone()
    custom_attn.k_attn.weight.data = k_weight.T.clone()
    custom_attn.v_attn.weight.data = v_weight.T.clone()
 
    custom_attn.q_attn.bias.data = q_bias.clone()
    custom_attn.k_attn.bias.data = k_bias.clone()
    custom_attn.v_attn.bias.data = v_bias.clone()
    custom_attn.c_proj.weight.data = c_proj_weight.T.clone()
    custom_attn.c_proj.bias.data = c_proj_bias.clone()

    block.attn = custom_attn

In [4]:
for block_flash in model_flash.transformer.h:
    c_attn_weight = block_flash.attn.c_attn.weight.clone()
    c_attn_bias = block_flash.attn.c_attn.bias.clone()
    c_proj_weight = block_flash.attn.c_proj.weight.clone()
    c_proj_bias = block_flash.attn.c_proj.bias.clone()
    
    q_weight, k_weight, v_weight = torch.chunk(c_attn_weight, chunks=3, dim=1)
    q_bias, k_bias, v_bias = torch.chunk(c_attn_bias, chunks=3, dim=0)

    custom_attn = CustomGPT2FlashAttention(nx=config.n_embd, n_ctx=config.n_ctx, config=config)
    custom_attn.q_attn.weight.data = q_weight.T.clone()
    custom_attn.k_attn.weight.data = k_weight.T.clone()
    custom_attn.v_attn.weight.data = v_weight.T.clone()
 
    custom_attn.q_attn.bias.data = q_bias.clone()
    custom_attn.k_attn.bias.data = k_bias.clone()
    custom_attn.v_attn.bias.data = v_bias.clone()
    custom_attn.c_proj.weight.data = c_proj_weight.T.clone()
    custom_attn.c_proj.bias.data = c_proj_bias.clone()

    block_flash.attn = custom_attn

In [5]:
eos_token = tokenizer.eos_token_id

In [6]:
data = preprocess_text(dataset['train'], min_length=15)

In [7]:
dl_train = split_in_sequence(data, tokenizer, batch_size=4, overlap=64, context_length=256)

Token indices sequence length is longer than the specified maximum sequence length for this model (1038 > 1024). Running this sequence through the model will result in indexing errors


In [8]:
input_data, labels = dl_train.dataset.tensors

torch.save({"input_data": input_data, "labels": labels}, "preprocessed_data_256.pt")
print("Данные сохранены в preprocessed_data256.pt")

Данные сохранены в preprocessed_data256.pt


In [6]:
from torch.utils.data import TensorDataset, DataLoader

# Загрузка данных с диска
saved_data = torch.load("preprocessed_data_256.pt")
input_data, labels = saved_data["input_data"], saved_data["labels"]

# Создание DataLoader
batch_size = 6  # Укажи тот же batch_size, который использовался при сохранении
dataset = TensorDataset(input_data, labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)

print("Данные успешно загружены и готовы к использованию.")

/tmp/ipykernel_19553/3939054396.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  saved_data = torch.load("preprocessed_data_256.pt")


Данные успешно загружены и готовы к использованию.


In [7]:
a, b = next(iter(dataloader))
a.shape, b.shape, len(dataloader)

(torch.Size([6, 256]), torch.Size([6, 256]), 95329)

In [8]:
from transformers import Adafactor, AdamW

optimizer_flash = Adafactor(
    model_flash.parameters(),
    scale_parameter=True,
    relative_step=True,
    warmup_init=True,
    lr=None
)

optimizer_vanilla = Adafactor(
    model_vanilla.parameters(),
    scale_parameter=True,
    relative_step=True,
    warmup_init=True,
    lr=None
)

In [9]:
torch.cuda.empty_cache()  # Освободить кэш GPU

In [18]:
from tqdm import tqdm

model_vanilla = model_vanilla.to("cuda")

epochs = 10
for epoch in range(epochs):
    model_vanilla.train()
    total_loss = 0

    for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}"):
        inputs, labels = batch
        inputs, labels = inputs.to("cuda"), labels.to("cuda")

        optimizer_vanilla.zero_grad()

        outputs = model_vanilla(input_ids=inputs, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer_vanilla.step()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1} - Average Loss: {avg_loss:.4f}")

Epoch 1:   1%|          | 2237/285986 [09:25<19:55:17,  3.96it/s]


KeyboardInterrupt: 

In [10]:
from tqdm import tqdm
from torch.amp import GradScaler, autocast

model_flash = model_flash.to("cuda")
# Creates a GradScaler once at the beginning of training.
scaler = GradScaler()

epochs = 10
for epoch in range(epochs):
    model_flash.train()
    total_loss = 0

    for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}"):
        inputs, labels = batch

        optimizer_flash.zero_grad()

        with autocast(device_type='cuda', dtype=torch.float16):
            outputs = model_flash(input_ids=inputs.cuda(), labels=labels.cuda())
            loss = outputs.loss

        # Backward pass with GradScaler
        scaler.scale(loss).backward()
        scaler.step(optimizer_flash)
        scaler.update()
        # total_loss += scaler.unscale_(loss)
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1} - Average Loss: {avg_loss:.4f}")

Epoch 1:   0%|          | 1/95329 [00:00<23:11:38,  1.14it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 294.00 MiB. GPU 0 has a total capacity of 3.70 GiB of which 231.62 MiB is free. Including non-PyTorch memory, this process has 3.05 GiB memory in use. Of the allocated memory 2.61 GiB is allocated by PyTorch, and 331.90 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [11]:
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 2            |        cudaMalloc retries: 2         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   2674 MiB |   3028 MiB |  14126 MiB |  11452 MiB |
|       from large pool |   2660 MiB |   3015 MiB |  13963 MiB |  11303 MiB |
|       from small pool |     13 MiB |     15 MiB |    162 MiB |    149 MiB |
|---------------------------------------------------------------------------|
| Active memory         |   2674 MiB |   3028 MiB |  14126 MiB |  11452 MiB |
|       from large pool |   2660 MiB |   3015 MiB |  13963 MiB |

## Changing Attantion Block in GPT2 model.

In [7]:
model.use_flash_attn = True
model2 = model.to("cuda")
generated = model2.generate(
    inputs.input_ids.to("cuda"),
    max_length=40,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.9,
    repetition_penalty=1.5,
)
print(tokenizer.decode(generated[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


sobaka torch.Size([1, 1, 12, 64]) torch.Size([1, 12, 64])
sobaka torch.Size([1, 1, 12, 64]) torch.Size([1, 12, 64])
sobaka torch.Size([1, 1, 12, 64]) torch.Size([1, 12, 64])
sobaka torch.Size([1, 1, 12, 64]) torch.Size([1, 12, 64])
sobaka torch.Size([1, 1, 12, 64]) torch.Size([1, 12, 64])
sobaka torch.Size([1, 1, 12, 64]) torch.Size([1, 12, 64])
sobaka torch.Size([1, 1, 12, 64]) torch.Size([1, 12, 64])
sobaka torch.Size([1, 1, 12, 64]) torch.Size([1, 12, 64])
sobaka torch.Size([1, 1, 12, 64]) torch.Size([1, 12, 64])
sobaka torch.Size([1, 1, 12, 64]) torch.Size([1, 12, 64])
sobaka torch.Size([1, 1, 12, 64]) torch.Size([1, 12, 64])
sobaka torch.Size([1, 1, 12, 64]) torch.Size([1, 12, 64])
sobaka torch.Size([1, 1, 12, 64]) torch.Size([1, 12, 64])
sobaka torch.Size([1, 1, 12, 64]) torch.Size([1, 12, 64])
sobaka torch.Size([1, 1, 12, 64]) torch.Size([1, 12, 64])
sobaka torch.Size([1, 1, 12, 64]) torch.Size([1, 12, 64])
sobaka torch.Size([1, 1, 12, 64]) torch.Size([1, 12, 64])
sobaka torch.S

## How quantization works

In [31]:
import torch

# Исходные веса в FP32
weights = torch.randn((4, 4), dtype=torch.float32)

# Преобразование в INT4
scale = 15 / weights.abs().max()  # Масштабируем до диапазона INT4
weights_int4 = torch.round(weights * scale).clamp(-15, 15).to(torch.int8)

# Преобразование в INT6
scale = 31 / weights.abs().max()  # Масштабируем до диапазона INT6
weights_int6 = torch.round(weights * scale).clamp(-31, 31).to(torch.int8)


In [32]:
weights

tensor([[ 1.2927,  0.1660, -0.6576,  1.4048],
        [ 0.3357, -1.2038, -0.5292,  1.2458],
        [-0.0752,  0.0526,  1.4541,  0.2409],
        [ 0.2059, -0.1143, -0.9621, -1.1691]])

In [33]:
weights_int4

tensor([[ 13,   2,  -7,  14],
        [  3, -12,  -5,  13],
        [ -1,   1,  15,   2],
        [  2,  -1, -10, -12]], dtype=torch.int8)

In [34]:
weights_int6

tensor([[ 28,   4, -14,  30],
        [  7, -26, -11,  27],
        [ -2,   1,  31,   5],
        [  4,  -2, -21, -25]], dtype=torch.int8)

In [35]:
weights_restored = weights_int6 / scale
weights_restored

tensor([[ 1.3133,  0.1876, -0.6567,  1.4072],
        [ 0.3283, -1.2195, -0.5160,  1.2664],
        [-0.0938,  0.0469,  1.4541,  0.2345],
        [ 0.1876, -0.0938, -0.9850, -1.1726]])

In [36]:
weights_restored - weights

tensor([[ 0.0207,  0.0216,  0.0009,  0.0023],
        [-0.0073, -0.0158,  0.0132,  0.0207],
        [-0.0186, -0.0057,  0.0000, -0.0063],
        [-0.0182,  0.0205, -0.0229, -0.0035]])

In [1]:
import torch
device_type = "cuda"
torch.amp.autocast_mode.is_autocast_available(device_type)

True

In [2]:
scaler = torch.GradScaler()

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer


# Загрузи токенизатор и модель
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, local_files_only=True)
model = AutoModelForCausalLM.from_pretrained(model_path, local_files_only=True)

# Тест: генерация текста
input_text = "Once upon a time"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
output = model.generate(input_ids, max_length=50)

print(tokenizer.decode(output[0], skip_special_tokens=True))

OSError: Incorrect path_or_model_id: '~/.cache/huggingface/hub/models--openai-community--gpt2'. Please provide either the path to a local folder or the repo_id of a model on the Hub.